In [52]:
import numpy as np
import gym
import os
from progressbar import ProgressBar
from time import sleep

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import backend as K

env = gym.make('Breakout-v0')
state_size = env.observation_space.shape[0]
action_size = env.env.action_space.n

In [69]:
state = env.reset()
done = False
lives = -1

while not done:
    old_lives = lives
    env.render()
    action = np.random.randint(action_size)
    screen, reward, done, ale_lives = env.step(action)
    lives = ale_lives['ale.lives']
      
    if lives != old_lives or not np.isclose(reward, 0.0):
        print(reward, done, lives)
    
    lives = ale_lives['ale.lives']
    

0.0 False 5
1.0 False 5
0.0 False 4
1.0 False 4
0.0 False 3
0.0 False 2
0.0 False 1
0.0 True 0
